In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):

    def __init__(self):
        super(Net, self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1,6,3),  # input channel 1 单色  output 6 channel(feature) 3x3 square convolution
            nn.ReLU(),
            nn.MaxPool2d(2,2,0),


            nn.Conv2d(6,16,3),
            nn.ReLU(),
            nn.MaxPool2d(2),
        )

        self.fc = nn.Sequential(
            nn.Linear(16*6*6,120),    # 6 * 6 image dimension
            nn.ReLU(),
            nn.Linear(120,84),
            nn.ReLU(),
            nn.Linear(84,10)
        )

    def forward(self,x):
        out = self.cnn(x)
        out = out.view(out.size()[0],-1)   # -1 让电脑自动计算维数
        return self.fc(out)

In [2]:
net = Net()
print(net)

Net(
  (cnn): Sequential(
    (0): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=576, out_features=120, bias=True)
    (1): ReLU()
    (2): Linear(in_features=120, out_features=84, bias=True)
    (3): ReLU()
    (4): Linear(in_features=84, out_features=10, bias=True)
  )
)


In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())

10
torch.Size([6, 1, 3, 3])


In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[-0.0235,  0.1428,  0.1683,  0.0056, -0.0369,  0.0565,  0.0174,  0.0790,
         -0.0067,  0.0491]], grad_fn=<AddmmBackward>)


In [ ]:
net.zero_grad()    # 反向传播前要将梯度手动清零，因为pytorch会自动计算梯度
out.backward(torch.randn(1, 10))

In [24]:
output = net(input)
target = torch.randn(1,10)  # a dummy target, for example
#target = target.view(1, -1)  # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)
print(loss.grad_fn)  # MSELoss
print(loss.grad_fn.next_functions[0][0])  # Linear
print(loss.grad_fn.next_functions[0][0].next_functions[0][0])  # ReLU

tensor(0.6536, grad_fn=<MseLossBackward>)


In [30]:
# net.zero_grad()     # zeroes the gradient buffers of all parameters
#
# print('cnn grad before backward')
# print(net)
#
# loss.backward()
#
# print('cnn grad after backward')
# print(net)

In [31]:
# update weights
import torch.optim as optim

# create your optimizer
optimizer = optim.SGD(net.parameters(), lr=0.01)
nums_epoch = 15
for epoch in range(nums_epoch):
    # in your training loop:
    optimizer.zero_grad()   # zero the gradient buffers
    output = net(input)
    loss = criterion(output, target)
    loss.backward()
    optimizer.step()    # Does the update
